In [7]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import json
import skimage
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
MODEL_DIR

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

## Create Model and Load Trained Weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
class CustomDataset(utils.Dataset):

    def load_custom(self,dataset_dir, subset,class_t):
        self.add_class("dent", 1, "dent")
        self.add_class("scratch",2,"scratch")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        annotations1 = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
        annotations = list(annotations1.values())
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            polygons = [r['shape_attributes'] for r in a['regions'].values()]
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                class_t,
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons)


    def load_mask(self, image_id):
        image_info = self.image_info[image_id]
        if image_info["source"] != 'dent' or image_info["source"] != "scratch":
            return super(self.__class__, self).load_mask(image_id)
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)


    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "dent" or info["source"] == "scratch":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
# Training Dent dataset
dent_dataset_train = CustomDataset()
dent_dataset_train.load_custom("D:/lakehead/sem-2/cv/projects&extra/Project/customImages/dent/","train","dent")
dent_dataset_train.prepare()

# Training Scratch dataset
scratch_dataset_train = CustomDataset()
scratch_dataset_train.load_custom("D:/lakehead/sem-2/cv/projects&extra/Project/customImages/scratch","train","scratch")
scratch_dataset_train.prepare()

# Validation Dent dataset
dent_dataset_val = CustomDataset()
dent_dataset_val.load_custom("D:/lakehead/sem-2/cv/projects&extra/Project/customImages/dent/","val","dent")
dent_dataset_val.prepare()

# Validation Scratch dataset
scratch_dataset_val = CustomDataset()
scratch_dataset_val.load_custom("D:/lakehead/sem-2/cv/projects&extra/Project/customImages/scratch/","val","scratch")
scratch_dataset_val.prepare()

In [ ]:
# *** This training schedule is an example. Update to your needs ***
# Since we're using a very small dataset, and starting from
# COCO trained weights, we don't need to train too long. Also,
# no need to train all layers, just the heads should do it.
print("Training network heads")
try:
    model.train(dent_dataset_train, dent_dataset_val,learning_rate=config.LEARNING_RATE,epochs=10,layers='heads')
except Exception:
    pass